In [ ]:
#Import libraries.
#Authenticate to access Earth Engine.
#Initialize the Earth Engine module.
#Load the shapefile containing boundaries for districts and upazilas.
#Filter for a specific district using the shapefile.
#Add night light data using VIIRS Nighttime Day/Night Band.
#Clip the night light data to the specific district.
#Calculate the minimum and maximum night light intensity values in the clipped region.
#Define legend keys based on dynamic min and max values.
#Generate gradient colors.
#Reverse the gradient colors.
#Define a function to get the highest intensity and coordinates for each upazila.
#Apply the function to each upazila within the specific district.
#Select relevant properties for each upazila to export.
#Export the data to a CSV file in Google Drive.
#Start the export task.
#Add the specific district layer to the map.
#Add the night light layer to the map.
#Center the map on the specified district.
#Define a function to add markers for each upazila with the highest night light intensity.
#Get the list of features and add markers for each.
#Create a custom legend as HTML.
#Add the custom legend to the map.
#Display the map.

import ee
from google.colab import auth
import geemap
from ipyleaflet import Marker, Popup
from ipywidgets import HTML
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Authenticate to access Earth Engine
auth.authenticate_user()

# Initialize the Earth Engine module
ee.Initialize(project='ee-2june2024')

# Load the uploaded shapefile containing the updated boundaries for both districts and upazilas
boundaries2020 = ee.FeatureCollection("projects/ee-2june2024/assets/bgd_adm_03")

# Filter for a specific district using the shapefile
district_name = 'Narayanganj'  # Specify the district name here
specific_district = boundaries2020.filter(ee.Filter.eq('district', district_name))

# Add night light data (using VIIRS Nighttime Day/Night Band)
night_light = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG') \
  .filterDate('2023-03-01', '2023-03-31') \
  .select('avg_rad') \
  .median() \
  .clip(specific_district.geometry())  # Clip the night light data to the specific district

# Calculate the minimum and maximum night light intensity values in the clipped region
min_max = night_light.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=specific_district.geometry(),
    scale=500,
    maxPixels=1e9
)
min_value = min_max.get('avg_rad_min').getInfo() if min_max.get('avg_rad_min') else 0
max_value = min_max.get('avg_rad_max').getInfo() if min_max.get('avg_rad_max') else 60

# Define legend keys based on dynamic min and max values
num_legend_items = 11  # Number of legend items
step = (max_value - min_value) / (num_legend_items - 1)
legend_keys = [f"{min_value + i * step:.2f}" for i in range(num_legend_items)]

# Generate gradient colors
def generate_gradient_colors(num_colors):
    cmap = plt.get_cmap('plasma')  # You can choose other colormaps like 'plasma', 'inferno', 'magma', 'viridis' etc.
    gradient_colors = [mcolors.rgb2hex(cmap(i / num_colors)) for i in range(num_colors)]
    return gradient_colors

gradient_colors = generate_gradient_colors(num_legend_items)

# Reverse the gradient colors
gradient_colors.reverse()

# Function to get the highest intensity and coordinates for each upazila
def get_max_intensity(feature):
    # Find the maximum intensity value within the feature geometry
    max_value = night_light.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=feature.geometry(),
        scale=500,
        maxPixels=1e9
    ).get('avg_rad')

    # Convert the max_value to an image
    max_value_image = ee.Image.constant(max_value)

    # Find all pixels with the maximum intensity value
    max_points = night_light.eq(max_value_image).selfMask().reduceToVectors(
        geometry=feature.geometry(),
        scale=500,
        maxPixels=1e9,
        geometryType='centroid',
        eightConnected=False
    )

    return feature.set({
        'max_intensity': max_value,
        'max_points': max_points.geometry().coordinates()
    })

# Apply the function to each upazila within the specific district
district_with_intensity = specific_district.map(get_max_intensity)

# Export relevant properties for each upazila to a CSV file
export_data = district_with_intensity.select(['upazila', 'max_intensity', 'max_points'])

# Export the data to a CSV file in Google Drive
task = ee.batch.Export.table.toDrive(
    collection=export_data,
    description='Narayanganj_Upazila_NightLight',
    fileFormat='CSV',
    selectors=['upazila', 'max_intensity', 'max_points']
)

# Start the export task
task.start()

# Add layers to the map
Map = geemap.Map()
Map.addLayer(specific_district, {}, 'Narayanganj Upazilas')
Map.addLayer(night_light, {'min': min_value, 'max': max_value, 'palette': gradient_colors}, 'Night Lights')

# Center map on the specified district
Map.centerObject(specific_district, 10)

# Add markers for each upazila with highest night light intensity
def add_markers(feature):
    max_points = feature.get('max_points').getInfo()
    intensity = feature.get('max_intensity').getInfo()
    upazila_name = feature.get('upazila').getInfo()

    # Handle nested structure of coordinates
    if isinstance(max_points[0], list):  # Check if it's nested
        for point in max_points:
            lon, lat = point

            # Verify the point is within the specific district
            point_geom = ee.Geometry.Point(lon, lat)
            if specific_district.geometry().contains(point_geom).getInfo():
                # Create a marker
                marker = Marker(location=(lat, lon))

                # Add a popup
                popup = Popup(
                    location=(lat, lon),
                    child=HTML(f"<b>Upazila:</b> {upazila_name}<br><b>Max Intensity:</b> {intensity}<br><b>Coordinates:</b> ({lat:.6f}, {lon:.6f})"),
                    close_button=False,
                    auto_close=False,
                    close_on_escape_key=False
                )

                # Add the marker and popup to the map
                Map.add_layer(marker)
                Map.add_layer(popup)
    else:
        lon, lat = max_points

        # Verify the point is within the specific district
        point_geom = ee.Geometry.Point(lon, lat)
        if specific_district.geometry().contains(point_geom).getInfo():
            # Create a marker
            marker = Marker(location=(lat, lon))

            # Add a popup
            popup = Popup(
                location=(lat, lon),
                child=HTML(f"<b>Upazila:</b> {upazila_name}<br><b>Max Intensity:</b> {intensity}<br><b>Coordinates:</b> ({lat:.4f}, {lon:.4f})"),
                close_button=False,
                auto_close=False,
                close_on_escape_key=False
            )

            # Add the marker and popup to the map
            Map.add_layer(marker)
            Map.add_layer(popup)

# Get the list of features and add markers for each
features = district_with_intensity.getInfo()['features']
for feature in features:
    add_markers(ee.Feature(feature))

# Add dynamic legend to the map
legend_dict = {legend_keys[i]: gradient_colors[i] for i in range(num_legend_items)}
Map.add_legend(title="Night Light Intensity", legend_dict=legend_dict, position="bottomleft")

# Display the map
Map
